In [16]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

runs = np.array((0, 1, 2, 3, 4, 5, 6, 7, 8, 9))
p_values = np.array((0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1))
rho_values = np.array((0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.25, 0.26))
iterations = 5000

L = 50

mean = np.zeros((len(p_values), len(rho_values), len(runs), iterations))
CI_int = np.zeros((len(p_values), len(rho_values), len(runs), iterations, 2))
CI_output = np.zeros((len(p_values), len(rho_values), len(runs), iterations, 2))
CI_output_act = np.zeros((len(p_values), len(rho_values), len(runs), 2))
significance = np.zeros((len(p_values), len(rho_values), len(runs), iterations))
phi_value_act = np.zeros((len(p_values), len(rho_values), len(runs)))

for p_ix, p_value in enumerate(p_values):
    print(p_value)
    for rho_ix, rho_value in enumerate(rho_values):
        for run in runs:
            counter1 = np.zeros(iterations)
            phi_values = np.genfromtxt(f'./results/raw_data/run_{int(run)}_p_{p_value}_rho_{rho_value}_L_{L}_iters_{iterations}.csv', delimiter = ',', skip_header=1)
            phi_values = phi_values[:,1]
            phi_values = phi_values
            for i in range(iterations):
                if i > 1000 and i < 4500:
                    slope, intercept, r_value, significance[p_ix, rho_ix, int(run), i], se = stats.linregress(list([np.linspace(0, 999, 1000), phi_values[i-500:i+500]]))
                    mean[p_ix, rho_ix, int(run), i] = np.mean(phi_values[i-500:i+500])
                    CI_int[p_ix, rho_ix, int(run), i] = np.percentile(phi_values[i-500:i+500], [5, 100])
                    CI_output[p_ix, rho_ix, int(run), i] = np.percentile(phi_values[i-500:i+500], [2.5, 97.5])
        
            for i in range(iterations):
                if i > 1000 and i < 4500:
                    # If the slope is not distinguishable from 0

                    if significance[p_ix, rho_ix, int(run), i] > 0.05:
                        counter0 = 0
                        # Count number of outliers seen by iteration i
                        for j in range(1000):
                            if phi_values[i+j-499] < CI_int[p_ix, rho_ix, int(run), i, 0]:
                                counter0 += 1
                        counter1[i] = counter0
                    else:
                        counter1[i] = 9999
                else:
                    counter1[i] = 9999
        
            # Find the last index that saw the least amount of outliers
            teller1_inv = counter1[::-1]
            min_value_teller = np.min(teller1_inv)
            index_min_val = list(teller1_inv).index(min_value_teller)
            actual_index = iterations - index_min_val - 1

            if counter1[actual_index] == 9999:
                print(f'Error: for p = {p_value}, rho = {rho_value}, run = {run}, there is no steady state measured. \n \
                        We should probably lower the range of intervals we include in the linear regression \n \
                        or make the lower percentile boundary highe. Corresponding ')

            # Save the phi_value that respresents a point in our 3D plot.
            phi_value_act[p_ix, rho_ix, int(run)] = mean[p_ix, rho_ix, int(run), actual_index]
            CI_output_act[p_ix, rho_ix, int(run)] = CI_output[p_ix, rho_ix, int(run), actual_index]
            



0.05
[-0.17527476  0.28530589]
[-0.21347283  0.26207733]
[-0.20849847  0.26905334]
[-0.19305167  0.33189107]
[-0.11003861  0.37258687]


KeyboardInterrupt: 